# Source Extration for 2-Photon Calcium Imaging Data

In [ ]:
# Step 1: Motion Correction
# Step 2: Transform motion corrected memory map file into format suitable for cnmf method
# Step 3: Select spatial parameters
# Step 4: Fit cnmf model
# Step 5: Refit cnmf model
# Step 6: Explore the estimates class

## Setting Parameters for Source Extraction and Motion Correction for 2p 

## CNMF for Source Extraction

## Refitting CNMF Algorithm

## Exploring the Estimates Class